In [ ]:
!pip install faker psycopg2-binary
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime, timedelta

# ==========================================
# 1. CONFIGURATION & SETUP
# ==========================================
fake = Faker()
Faker.seed(67)
np.random.seed(67)

# Theater Configuration (The "Weighted Cities" Logic)
THEATERS = {
    'Tunis': {
        'id': 1,
        'name': 'CinéInsights Tunis (Capital)',
        'base_traffic': 150, # Avg tickets per day
        'base_price': 15.0,  # TND
        'capacity_factor': 1.0
    },
    'Sousse': {
        'id': 2,
        'name': 'CinéInsights Sousse (Pearl)',
        'base_traffic': 90,  # Avg tickets per day
        'base_price': 12.0,  # TND
        'capacity_factor': 0.6
    },
    'Sfax': {
        'id': 3,
        'name': 'CinéInsights Sfax (Industrial)',
        'base_traffic': 60,  # Avg tickets per day
        'base_price': 10.0,  # TND
        'capacity_factor': 0.4
    }
}

# Date Range (Matching your weather data)
START_DATE = datetime(2023, 12, 20)
END_DATE = datetime(2025, 12, 20)

# ==========================================
# 2. DATA LOADING & PREP
# ==========================================
print("Loading Source Data...")

# Load Movies
df_movies = pd.read_csv('filtered_tmdb_movies_sample.csv')
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'])
# Clean genres (take the first one as primary for logic simplicity)
df_movies['primary_genre'] = df_movies['genres'].apply(lambda x: x.split(',')[0] if pd.notnull(x) else 'Drama')

# Load Weather
df_weather = pd.read_csv('tunisia_historical_weather.csv')
df_weather['date'] = pd.to_datetime(df_weather['date'])

print(f"Loaded {len(df_movies)} movies and {len(df_weather)} weather records.")

# ==========================================
# 3. HELPER FUNCTIONS (The "Business Logic")
# ==========================================

def get_day_weight(date_obj):
    """
    Weighted Days: Weekend multiplier.
    Fri(4), Sat(5), Sun(6)
    """
    day_num = date_obj.weekday()
    if day_num == 4: return 1.3  # Friday
    if day_num == 5: return 1.5  # Saturday
    if day_num == 6: return 1.4  # Sunday
    return 0.8  # Weekdays are slower

def get_weather_impact(weather_row):
    """
    Weather Logic:
    Rain/Cold -> People go to cinema (+ Impact)
    Hot/Clear -> People go to beach (- Impact)
    """
    if str(weather_row['weather_state']).strip() in ['Rainy', 'Stormy', 'Cold']:
        return 1.25 # Boost
    elif str(weather_row['temp_category']).strip() == 'Hot' and str(weather_row['weather_state']).strip() == 'Clear':
        return 0.75 # Penalty (Beach day)
    return 1.0 # Neutral

def get_price_elasticity(base_price, movie_rating):
    """
    Price Logic:
    Higher rating (> 7.5) = Premium Price.
    Higher price = Slightly lower purchase probability (Elasticity).
    """
    final_price = base_price
    if movie_rating > 7.5:
        final_price += 2.0 # Premium for good movies

    # Simple elasticity: higher price -> slightly lower volume factor
    elasticity_factor = 1.0
    if final_price > 14:
        elasticity_factor = 0.9

    return final_price, elasticity_factor

def generate_customer_profile(genre):
    """
    Generates a customer based on Movie Genre assumptions.
    """
    # 0 = Male, 1 = Female
    gender_prob = 0.5
    age_min, age_max = 16, 60

    if genre in ['Action', 'Thriller', 'War', 'Crime']:
        gender_prob = 0.3 # More Male
        age_min, age_max = 18, 45
    elif genre in ['Romance', 'Drama']:
        gender_prob = 0.7 # More Female
        age_min, age_max = 20, 50
    elif genre in ['Animation', 'Family']:
        gender_prob = 0.5
        age_min, age_max = 12, 45 # Families
    elif genre in ['Horror']:
        gender_prob = 0.4
        age_min, age_max = 16, 30

    sex = 'F' if np.random.random() < gender_prob else 'M'
    age = np.random.randint(age_min, age_max)
    name = fake.name_female() if sex == 'F' else fake.name_male()

    return name, sex, age

# ==========================================
# 4. DATA GENERATION LOOP
# ==========================================
customers_list = []
sales_list = []
customer_id_counter = 1001
transaction_id_counter = 50001

current_date = START_DATE
print("Starting Data Generation (This may take a minute)...")

while current_date <= END_DATE:
    date_str = current_date.strftime('%Y-%m-%d')

    # Get movies active (Assume movies screen for 4 weeks after release, roughly)
    # Since dataset is historic, we just pick random subset of 8 movies per day for variety
    daily_movies = df_movies.sample(n=8)

    for city_key, city_data in THEATERS.items():
        # 1. Get Weather for this City/Date
        w_row = df_weather[
            (df_weather['city'] == city_key) &
            (df_weather['date'] == current_date)
        ]

        if w_row.empty:
            weather_mult = 1.0
        else:
            weather_mult = get_weather_impact(w_row.iloc[0])

        # 2. Calculate Traffic Volume
        day_mult = get_day_weight(current_date)
        base_vol = city_data['base_traffic']

        # Total expected tickets for this city today
        daily_potential = int(base_vol * day_mult * weather_mult)

        # Distribute potential across the 8 movies
        for _, movie in daily_movies.iterrows():
            # Runtime Logic: Long movies (>150m) have fewer screenings -> 20% less capacity
            runtime_penalty = 0.8 if movie['runtime'] > 150 else 1.0

            # Price Calculation
            price, elasticity = get_price_elasticity(city_data['base_price'], movie['vote_average'])

            # Rating Impact (Better movies sell more)
            rating_mult = 1.0 + ((movie['vote_average'] - 6.0) / 10.0) # slightly boosts if > 6

            # Calculate final tickets for this movie
            tickets_for_movie = int(
                (daily_potential / 8) * # Share of daily traffic
                runtime_penalty *
                elasticity *
                rating_mult *
                np.random.uniform(0.8, 1.2) # Random noise
            )

            # Generate Transactions
            for _ in range(tickets_for_movie):
                # 80% chance it's an existing customer, 20% new
                if len(customers_list) > 0 and np.random.random() > 0.2:
                    cust_id = random.choice(customers_list)['customer_id']
                else:
                    # New Customer
                    c_name, c_sex, c_age = generate_customer_profile(movie['primary_genre'])
                    cust_id = customer_id_counter
                    customers_list.append({
                        'customer_id': cust_id,
                        'name': c_name,
                        'gender': c_sex,
                        'age': c_age,
                        'email': f"user{cust_id}@example.com",
                        'city': city_key, # Customer lives in this city
                        'created_at': date_str
                    })
                    customer_id_counter += 1

                # Create Sale Record
                sales_list.append({
                    'transaction_id': transaction_id_counter,
                    'date_key': date_str, # Connecting to Time Dimension
                    'customer_id': cust_id,
                    'movie_id': movie['id'],
                    'theater_id': city_data['id'],
                    'city': city_key,
                    'ticket_price': round(price, 2),
                    'quantity': 1, # Simplified to 1 ticket per trans for granularity
                    'total_amount': round(price, 2)
                })
                transaction_id_counter += 1

    current_date += timedelta(days=1)

# ==========================================
# 5. EXPORT
# ==========================================
df_customers = pd.DataFrame(customers_list)
df_sales = pd.DataFrame(sales_list)

print(f"Generation Complete!")
print(f"Total Customers Created: {len(df_customers)}")
print(f"Total Transactions Created: {len(df_sales)}")

# Save to CSV for the next Phase (ETL)
df_customers.to_csv('app_customers.csv', index=False)
df_sales.to_csv('app_sales.csv', index=False)
print("Files saved: 'app_customers.csv' and 'app_sales.csv'")

In [ ]:
!pip install psycopg2-binary sqlalchemy

import pandas as pd
from sqlalchemy import create_engine, text

# ==========================================
# 1. DATABASE CONNECTION CONFIG
# ==========================================
# REPLACE WITH YOUR GOOGLE CLOUD SQL DETAILS
db_config = {
    'user': 'cinema_admin',          # Default user is usually 'postgres'
    'password': '<DB_PASSWORD>',    # replaced sensitive value
    'host': '<DB_HOST>',            # replaced sensitive value
    'port': '5432',
    'dbname': 'cinema_db'         # Database name (create a specific one if you want)
}

# Create the connection string
connection_str = f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}"
engine = create_engine(connection_str)

# ==========================================
# 3. UPLOAD DATA
# ==========================================

# 3.1 Load CSVs generated in the previous step
try:
    df_customers = pd.read_csv('app_customers.csv')
    df_sales = pd.read_csv('app_sales.csv')

    # 3.2 Create the Theaters DataFrame manually (since it wasn't a CSV)
    theaters_data = [
        {'theater_id': 1, 'name': 'CinéInsights Tunis (Capital)', 'city': 'Tunis', 'capacity_factor': 1.0},
        {'theater_id': 2, 'name': 'CinéInsights Sousse (Pearl)', 'city': 'Sousse', 'capacity_factor': 0.6},
        {'theater_id': 3, 'name': 'CinéInsights Sfax (Industrial)', 'city': 'Sfax', 'capacity_factor': 0.4}
    ]
    df_theaters = pd.DataFrame(theaters_data)

    # 3.3 Upload to SQL (if_exists='append' adds data to the tables we just made)
    print("⏳ Uploading Customers... (This might take a moment)")
    df_customers.to_sql('app_customers', engine, if_exists='append', index=False, method='multi', chunksize=1000)

    print("⏳ Uploading Theaters...")
    df_theaters.to_sql('app_theaters', engine, if_exists='append', index=False)

    print("⏳ Uploading Sales... (This is the big one)")
    df_sales.to_sql('app_sales', engine, if_exists='append', index=False, method='multi', chunksize=1000)

    print("🎉 SUCCESS! All data uploaded to PostgreSQL.")

except Exception as e:
    print(f"❌ Error during data upload: {e}")

✅ Tables `app_customers`, `app_theaters`, and `app_sales` created successfully.
⏳ Uploading Customers... (This might take a moment)
⏳ Uploading Theaters...
⏳ Uploading Sales... (This is the big one)
🎉 SUCCESS! All data uploaded to PostgreSQL.


In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

# ==========================================
# 1. SETUP CONNECTION (Same as before)
# ==========================================
# Re-use the config from the previous step
db_config = {
    'user': 'cinema_admin',          # Default user is usually 'postgres'
    'password': '<DB_PASSWORD>',    # replaced sensitive value
    'host': '<DB_HOST>',            # replaced sensitive value
    'port': '5432',
    'dbname': 'cinema_db'         # Database name (create a specific one if you want)
}
connection_str = f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}"
engine = create_engine(connection_str)

def run_test(title, query):
    print(f"\n--- TEST: {title} ---")
    try:
        df = pd.read_sql(query, engine)
        if df.empty:
            print("⚠️ Result is empty.")
        else:
            print(df.to_markdown(index=False)) # Markdown format looks good in Colab logs
    except Exception as e:
        print(f"❌ Error: {e}")

# ==========================================
# 2. EXECUTE TESTS
# ==========================================

# TEST 1: Basic Row Counts
# Goal: Ensure we have thousands of rows, not zero.
query_counts = """
SELECT
    (SELECT COUNT(*) FROM app_customers) as total_customers,
    (SELECT COUNT(*) FROM app_sales) as total_sales,
    (SELECT COUNT(*) FROM app_theaters) as total_theaters;
"""
run_test("Row Counts", query_counts)


# TEST 2: Verify "Weighted Cities" Logic
# Goal: Tunis should have significantly more sales than Sfax.
# Logic from generator: Tunis (High), Sousse (Med), Sfax (Low)
query_city_logic = """
SELECT
    city,
    COUNT(*) as transaction_count,
    SUM(total_amount) as total_revenue
FROM app_sales
GROUP BY city
ORDER BY transaction_count DESC;
"""
run_test("City Logic Verification (Tunis should be top)", query_city_logic)


# TEST 3: Verify "Weighted Days" Logic
# Goal: Friday/Saturday/Sunday should be higher than Monday/Tuesday.
query_day_logic = """
SELECT
    TO_CHAR(date_key, 'Day') as day_name,
    COUNT(*) as transaction_count
FROM app_sales
GROUP BY TO_CHAR(date_key, 'Day'), EXTRACT(ISODOW FROM date_key)
ORDER BY EXTRACT(ISODOW FROM date_key);
"""
run_test("Day of Week Logic (Fri-Sun should be highest)", query_day_logic)


# TEST 4: Price & Movie Rating Correlation
# Goal: See if expensive tickets correlate with higher rated movies.
# We join sales with your movies CSV (conceptually, but here we check the price distribution)
query_price_check = """
SELECT
    ticket_price,
    COUNT(*) as sales_volume
FROM app_sales
GROUP BY ticket_price
ORDER BY ticket_price DESC
LIMIT 10;
"""
run_test("Top Ticket Prices (Check for Premium Pricing)", query_price_check)


# TEST 5: Data Integrity Check
# Goal: Ensure no negative prices or null dates.
query_integrity = """
SELECT * FROM app_sales
WHERE ticket_price < 0 OR date_key IS NULL
LIMIT 5;
"""
run_test("Integrity Check (Should be Empty)", query_integrity)


--- TEST: Row Counts ---
|   total_customers |   total_sales |   total_theaters |
|------------------:|--------------:|-----------------:|
|             48031 |        241418 |                3 |

--- TEST: City Logic Verification (Tunis should be top) ---
| city   |   transaction_count |    total_revenue |
|:-------|--------------------:|-----------------:|
| Tunis  |              117062 |      1.80674e+06 |
| Sousse |               75629 | 940448           |
| Sfax   |               48727 | 508424           |

--- TEST: Day of Week Logic (Fri-Sun should be highest) ---
| day_name   |   transaction_count |
|:-----------|--------------------:|
| Monday     |               25289 |
| Tuesday    |               25837 |
| Wednesday  |               26347 |
| Thursday   |               25582 |
| Friday     |               43261 |
| Saturday   |               49610 |
| Sunday     |               45492 |

--- TEST: Top Ticket Prices (Check for Premium Pricing) ---
|   ticket_price |   sales_